In [1]:
import json
data_dir = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_clipped/"+'notes.json'
queries = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_queries/"+'notes.json'


In [2]:
def splitNotes(notes):
    new_notes = []
    for note in notes:
        start = note[1]/1000.
        end = note[2]/1000.
        startSec = int(start)
        endSec = int(end)

        if startSec == endSec:
            new_notes.append(note)
        else:
            new_notes.append([note[0], note[1], (startSec+1)*1000, note[3]])
            for i in range(startSec+1, endSec-1):
                new_notes.append([note[0], int(i*1000), int(i+1)*1000, note[3]])
            new_notes.append([note[0], int(endSec*1000), note[2], note[3]])
    return new_notes


def getStrides(notes, size):
    notes = sorted(notes, key=lambda x: x[1])
    chunks = []
    for _ in range(len(notes)):
        chunks.append([])

    curT = 0
    nextidx = 0
    flag = False

    i = 0
    while(i < len(notes)):
        st = notes[i][1]
        maxT = curT + size
        if st >= maxT:
            curT += 1000
            i = nextidx
            flag = False
            continue

        if st % 1000 == 0 and flag == False and st != curT:
            nextidx = i
            flag = True

        pos = int(curT/1000)
        chunks[pos].append(notes[i])
        i += 1
    ans = []
    for c in chunks:
        if len(c) > 0:
            ans.append(c)
    return ans


In [3]:
with open(data_dir, 'r') as f:
    fname_to_notes = json.load(f)


In [10]:
midiEt_to_note = {
    12: "C",
    13: "C#",
    14: "D",
    15: "D#",
    16: "E",
    17: "F",
    18: "F#",
    19: "G",
    20: "G#",
    21: "A",
    22: "A#",
    23: "B"
}


# a function which converts a sequence of midiEt to a sequence of notes
def midiEt_to_note_sequence(midiEt_sequence):
    note_sequence = []
    for midiEt in midiEt_sequence:
        num = midiEt % 12
        note_sequence.append(midiEt_to_note[num + 12])
    return "".join(note_sequence)


In [49]:
from mongeau_sankoff import similarity, Note
import miditoolkit as mtk
# queries=[Note("G",2,rest=False)]

# t=total_time/(4*tempo)
# duration=int(end-start/t)

In [50]:
obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid")

In [51]:
notes = obj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

In [52]:
rep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        rep.append(Note("C",int((n.start-prev_end)),rest=True))
    rep.append(Note(midiEt_to_note[n.pitch%12 + 12],int((n.end-n.start)),rest=False))

In [53]:
qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_query_melody.mid")


In [54]:
notes = obj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

In [55]:
qrep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        qrep.append(Note("C", int((n.start-prev_end)), rest=True))
    qrep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)), rest=False))


In [56]:
similarity(qrep, rep)

KeyboardInterrupt: 

In [36]:
import music21


def getScale(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.mode)


In [41]:
getScale(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid")

'major'

In [40]:
getScale(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_query_melody.mid")


'major'